In [3]:
import setup
setup.init_django(project_name='lytehome')

In [5]:
import helpers

In [7]:
url = "https://www.amazon.com/Playstation-PS5-Console-Fortnite-Digital-5/dp/B0DJHG2VVS/ref=sr_1_1?crid=7JUA46RHH9I1&dib=eyJ2IjoiMSJ9.2HIIwJJ2kFJOQsSo4Cr-3UZqiUiWI34lHk30YqvJcPQFta3uFinBWXn8ciCu2QS6d7UJKjwebgYumcQceGMkzhhvan2Qra8STM5yBjBxwN1oCUZJRNWDomRJyTbrm-bnIbMfWT1tjnNb6pz6-H7UhXMFcIpFyA9TJqWLAyZPmkCBZoPk3EdZd_3WbbAm9Wwp3Rc4fqBYd808YJg7IBhUmIX2uxnMpKlRhhQz4hZW7as.s9RgsUvDFAYop42BKQC8SqsXNYaJe-JvtOfDyRPGAnc&dib_tag=se&keywords=ps5&qid=1734110304&sprefix=p%2Caps%2C528&sr=8-1&th=1"

In [8]:
from bs4 import BeautifulSoup

In [10]:
html = helpers.scrape(url=url)

Connecting to Scraping Browser...
Connected! Navigating...https://www.amazon.com/Playstation-PS5-Console-Fortnite-Digital-5/dp/B0DJHG2VVS/ref=sr_1_1?crid=7JUA46RHH9I1&dib=eyJ2IjoiMSJ9.2HIIwJJ2kFJOQsSo4Cr-3UZqiUiWI34lHk30YqvJcPQFta3uFinBWXn8ciCu2QS6d7UJKjwebgYumcQceGMkzhhvan2Qra8STM5yBjBxwN1oCUZJRNWDomRJyTbrm-bnIbMfWT1tjnNb6pz6-H7UhXMFcIpFyA9TJqWLAyZPmkCBZoPk3EdZd_3WbbAm9Wwp3Rc4fqBYd808YJg7IBhUmIX2uxnMpKlRhhQz4hZW7as.s9RgsUvDFAYop42BKQC8SqsXNYaJe-JvtOfDyRPGAnc&dib_tag=se&keywords=ps5&qid=1734110304&sprefix=p%2Caps%2C528&sr=8-1&th=1
Taking page screenshot to file page.png
Navigated! Scraping page content...


In [11]:
soup = BeautifulSoup(html)

In [19]:
def find_product_table_data(html):
    soup = BeautifulSoup(html, "html.parser")
    product_data = soup.find('div', id='prodDetails')
    if product_data is None:
         return []
    table = product_data.find('table')
    columns = [f"{x.text}".strip() for x in table.find_all('th')]
    table_data=[]
    for i, row in enumerate(table.find_all('tr')):  # [1:] to skip the header row
        # Get all cells in the row
        cells = row.find_all('td')
        # Create a dictionary for the current row, mapping header to cell data
        row_data = {columns[i]: f'{cell.text}'.strip() for cell in cells}
        # Add the dictionary to your list
        table_data.append(row_data)
    return table_data

In [20]:
def find_product_rating(html):
    soup = BeautifulSoup(html, "html.parser")
    average_rating = soup.find(id='averageCustomerReviews').find_all("span", class_='a-size-base')[0].text.strip()
    average_rating = "".join([x for x in f"{average_rating}".strip() if x.isdigit() or x == '.'])
    average_rating = float(average_rating)
    rating_data = soup.find(id='acrCustomerReviewText').text
    rating_count = int(''.join([x for x in rating_data if x.isdigit()]))
    rating_count
    return {
        'average': average_rating,
        'count': rating_count,
    }

In [21]:
def extract_amazon_product_data(html):
    soup = BeautifulSoup(html, "html.parser")
    productTitle = soup.find('span', id='productTitle')
    productTitleText = f"{productTitle.text}".strip()
    productPrice = soup.find_all('span', class_='a-price-whole')[0]
    productPrice = f"{productPrice.text}".strip()
    productPriceText = "".join([x for x in productPrice if x.isdigit() or x == '.'])
    productPriceNum = float(productPriceText)
    try:
        productDescription = soup.find('div', id='productDescription').text
    except:
        productDescription = ''
    featureBullets = soup.find('div', id='feature-bullets').text
    asin = ''
    metadata_items = find_product_table_data(html)
    for data in metadata_items:
        if data.get("ASIN") is None:
            continue
        else:
            asin = data.get("ASIN")
            break
    return {
        'asin': asin,
        'title': productTitleText,
        'price_raw': productPrice,
        'price_text': productPriceText,
        'price': productPriceNum,
        'metadata': metadata_items,
        'description': productDescription,
        'feature_bullets': featureBullets,
        'rating': find_product_rating(html)
    }

In [22]:
print(extract_amazon_product_data(html))

{'asin': 'B0DJHG2VVS', 'title': 'Playstation PS5 Console - Fortnite Cobalt Star Digital Edition', 'price_raw': '73.', 'price_text': '73.', 'price': 73.0, 'metadata': [{'ASIN': 'B0DJHG2VVS'}, {'Release date': 'November 22, 2024'}, {'Customer Reviews': '4.7  4.7 out of 5 stars    \n    8,847 ratings   \n\n\n 4.7 out of 5 stars'}, {'Best Sellers Rank': '#252 in Video Games (See Top 100 in Video Games)  #4 in PlayStation 5 Consoles'}, {'Product Dimensions': '14.6 x 17 x 5.1 inches; 8.96 Pounds'}, {'Type of item': 'Video Game'}, {'Language': 'English'}, {'Rated': 'Teen'}, {'Item Weight': '8.96 pounds'}, {'Manufacturer': 'Sony Interactive Entertainment'}, {'Batteries': '1 Lithium Ion batteries required. (included)'}, {'Date First Available': 'November 22, 2024'}], 'description': ' \n Upgrade your Fortnite experience with the PS5® Digital Edition - Fortnite® Cobalt Star Bundle. This bundle comes with a PS5 console + 8 in-game cosmetics (estimated value of 5,000 V-Bucks) and 1,000 V-Bucks for 